In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
left_img = cv2.imread('left.png')
right_img = cv2.imread('right.png')

In [ ]:
def compute_disparity_map(left_img, right_img, window_size=5, max_disparity=64):
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    height, width = left_gray.shape
    disparity_map = np.zeros((height, width), np.float32)
    for y in range(window_size, height - window_size):
        for x in range(window_size, width - window_size):
            best_offset = 0
            min_sad = float('inf')
            left_block = left_gray[y - window_size:y + window_size + 1, x - window_size:x + window_size + 1]
            for d in range(max_disparity):
                if x - d - window_size < 0:
                    continue
                right_block = right_gray[y - window_size:y + window_size + 1, x - d - window_size:x - d + window_size + 1]
                sad = np.sum(np.abs(left_block.astype(np.float32) - right_block.astype(np.float32)))
                if sad < min_sad:
                    min_sad = sad
                    best_offset = d
            disparity_map[y, x] = best_offset
    disparity_normalized = cv2.normalize(disparity_map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    disparity_normalized = np.uint8(disparity_normalized)
    disparity_normalized = 255 - disparity_normalized
    return disparity_normalized
disparity_map = compute_disparity_map(left_img, right_img)

In [ ]:
def generate_depth_heatmap(disparity_map):
    heatmap = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)
    return heatmap
depth_heatmap = generate_depth_heatmap(disparity_map)
cv2.imwrite('depth.png', depth_heatmap)
plt.imshow(depth_heatmap[..., ::-1])
plt.axis('off')
plt.show()